In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [4]:
df_train = pd.read_csv(train_file_path, sep="\t", header=None, names=["class", "message"])
df_train

,class,message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...


In [5]:
df_test = pd.read_csv(test_file_path, sep="\t", header=None, names=["class", "message"])
df_test

,class,message
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
...,...,...
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...


In [6]:
df_train["class"] = pd.factorize(df_train["class"])[0]
df_test["class"] = pd.factorize(df_test["class"])[0]

In [7]:
df_train

,class,message
0,0,ahhhh...just woken up!had a bad dream about u ...
1,0,you can never do nothing
2,0,"now u sound like manky scouse boy steve,like! ..."
3,0,mum say we wan to go then go... then she can s...
4,0,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,0,just woke up. yeesh its late. but i didn't fal...
4175,0,what do u reckon as need 2 arrange transport i...
4176,1,free entry into our £250 weekly competition ju...
4177,1,-pls stop bootydelious (32/f) is inviting you ...


In [8]:
df_test

,class,message
0,0,i am in hospital da. . i will return home in e...
1,0,"not much, just some textin'. how bout you?"
2,0,i probably won't eat at all today. i think i'm...
3,0,don‘t give a flying monkeys wot they think and...
4,0,who are you seeing?
...,...,...
1387,0,true dear..i sat to pray evening and felt so.s...
1388,0,"what will we do in the shower, baby?"
1389,0,where are you ? what are you doing ? are yuou ...
1390,1,ur cash-balance is currently 500 pounds - to m...


In [37]:
train_labels =  df_train["class"].values
train_ds = tf.data.Dataset.from_tensor_slices((df_train["message"].values, train_labels))

In [38]:
test_labels =  df_test["class"].values
test_ds = tf.data.Dataset.from_tensor_slices((df_test["message"].values, test_labels))

In [39]:
BUFFER_SIZE = 100
BATCH_SIZE = 32
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [40]:
vec = tf.keras.layers.TextVectorization(output_mode='int', max_tokens=1000, output_sequence_length=1000)

vec.adapt(train_ds.map(lambda text, label: text))

In [41]:
vec.vocabulary_size()

1000

In [42]:
vocab = np.array(vec.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'to', 'i', 'you', 'a', 'the', 'u', 'and', 'in', 'is',
       'me', 'my', 'for', 'your', 'of', 'it', 'call', 'have', 'on'],
      dtype='<U15')

In [43]:
model = tf.keras.Sequential([
    vec,
    tf.keras.layers.Embedding(
        len(vec.get_vocabulary()),
        64,
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])


model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy'],
)

In [45]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    validation_steps=30,
    epochs=10,
    verbose=1
)

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 16s 118ms/step - accuracy: 0.8714 - loss: 0.2833 - val_accuracy: 0.8604 - val_loss: 0.2165
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 20s 114ms/step - accuracy: 0.9125 - loss: 0.1839 - val_accuracy: 0.9560 - val_loss: 0.1388
Epoch 3/10
  1/131 ━━━━━━━━━━━━━━━━━━━━ 12s 99ms/step - accuracy: 0.9688 - loss: 0.0532

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


131/131 ━━━━━━━━━━━━━━━━━━━━ 15s 115ms/step - accuracy: 0.9686 - loss: 0.1053 - val_accuracy: 0.9677 - val_loss: 0.1096
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 14s 110ms/step - accuracy: 0.9798 - loss: 0.0743 - val_accuracy: 0.9676 - val_loss: 0.0801
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 115ms/step - accuracy: 0.9841 - loss: 0.0592 - val_accuracy: 0.9698 - val_loss: 0.0864
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 40s 263ms/step - accuracy: 0.9873 - loss: 0.0483 - val_accuracy: 0.9745 - val_loss: 0.0780
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 22s 115ms/step - accuracy: 0.9899 - loss: 0.0410 - val_accuracy: 0.9812 - val_loss: 0.0760
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 15s 114ms/step - accuracy: 0.9926 - loss: 0.0344 - val_accuracy: 0.9838 - val_loss: 0.0634
Epoch 9/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 116ms/step - accuracy: 0.9924 - loss: 0.0315 - val_accuracy: 0.9812 - val_loss: 0.0728
Epoch 10/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 20s 111ms/step - accuracy: 0.9951 - loss: 0.0256 - va

In [46]:
test_loss, test_acc = model.evaluate(test_ds)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.9842 - loss: 0.0644
Test Loss: 0.06934268027544022
Test Accuracy: 0.9820402264595032


In [47]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text = tf.expand_dims(pred_text, axis=0)
  ps = model.predict([pred_text])
  print(ps)
  p = ps[0][0]
  return [p, "ham" if p <0.5 else "spam"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step
[[-5.920948]]
[-5.920948, 'ham']


In [48]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
[[-5.920948]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
[[1.4505488]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
[[-6.1689086]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
[[5.327883]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
[[5.494821]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
[[-6.0517135]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
[[-5.6575346]]
You passed the challenge. Great job!
